In [ ]:
# !pip install llama-index

# !pip install llama-index-embeddings-huggingface
# !pip install google-generativeai>=0.3.0 llama-index-embeddings-gemini
# !pip install llama-index-llms-cohere
# !pip install llama-index-embeddings-cohere

# !pip install llama-index-llms-groq

In [1]:
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    ServiceContext, 
    load_index_from_storage
)
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.groq import Groq

c:\Users\Rahul Gupta\anaconda3\envs\condarag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
cwd = os.getcwd()
print(f'Current Working Directory: {cwd}')

Current Working Directory: c:\Users\Rahul Gupta\Documents\RG\GenAI\0.self_explore\custom_rag_llamaindex


In [24]:
# Data Ingestion
# reader = SimpleDirectoryReader(input_dir=f".\\documents\\coe_demo")

# reader = SimpleDirectoryReader(input_files=["path/to/file1", "path/to/file2"])
reader = SimpleDirectoryReader(input_files=[f".\\documents\\coe_demo\\data-science-lifecycle-ebook.pdf"])

print(f"Reader: {reader}")
documents = reader.load_data()
print(f"Documents: {documents}")
print(f"Number of documents: {len(documents)}")

Reader: <llama_index.core.readers.file.base.SimpleDirectoryReader object at 0x000001BCD1767970>
Documents: [Document(id_='a91546ca-ecbe-4fba-9d20-6688c23f60df', embedding=None, metadata={'page_label': '1', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'documents\\coe_demo\\data-science-lifecycle-ebook.pdf', 'file_type': 'application/pdf', 'file_size': 7899425, 'creation_date': '2024-05-20', 'last_modified_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Lifecycle of  \nmachine learning models\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='ab354848-f3b7-4858-8b2c-b88208de9b8c', embedding=None, metadata=

In [5]:
documents[0]

Document(id_='1802367d-178b-47db-afd0-43e8e035e678', embedding=None, metadata={'page_label': '1', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'c:\\Users\\Rahul Gupta\\Documents\\RG\\GenAI\\0.self_explore\\custom_rag_llamaindex\\documents\\coe_demo\\data-science-lifecycle-ebook.pdf', 'file_type': 'application/pdf', 'file_size': 7899425, 'creation_date': '2024-05-20', 'last_modified_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Lifecycle of  \nmachine learning models\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [6]:
for i, doc in enumerate(documents):
    print(i, doc)
    print(i, doc.metadata)

0 Doc ID: 1802367d-178b-47db-afd0-43e8e035e678
Text: Lifecycle of   machine learning models
0 {'page_label': '1', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'c:\\Users\\Rahul Gupta\\Documents\\RG\\GenAI\\0.self_explore\\custom_rag_llamaindex\\documents\\coe_demo\\data-science-lifecycle-ebook.pdf', 'file_type': 'application/pdf', 'file_size': 7899425, 'creation_date': '2024-05-20', 'last_modified_date': '2024-01-29'}
1 Doc ID: 1f3b0ed1-ed90-4903-aef6-78209b2b086d
Text: 1 2/Introduction   Steps of building machine learning models  2/
Machine learning is an area that enterprises  are increasingly
investing in or identifying as a potential area of growth. There are
many reasons enterprises invest in machine learning, from being able
to leverage data to find insights about their customers to making
processes more ...
1 {'page_label': '2', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'c:\\Users\\Rahul Gupta\\Documents\\RG\\GenAI\\0.self_explore\\custom_rag

In [ ]:
# Vector Embeddings

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en-v1.5"
# )

In [8]:
# from llama_index.embeddings.gemini import GeminiEmbedding
# embed_model = GeminiEmbedding(model_name="models/embedding-001")

In [ ]:
from llama_index.embeddings.cohere import CohereEmbedding

# with input_typ='search_query'
embed_model = CohereEmbedding(
    cohere_api_key=os.getenv('COHERE_API_KEY'),
    model_name="embed-english-v3.0",
    input_type="search_document" # "search_query", "search_document"
)

# embeddings = embed_model.get_text_embedding("Hello CohereAI!")
# embeddings

In [25]:
# Chunking
# Instead of chunking text with a fixed chunk size, the semantic splitter adaptively picks the breakpoint 
# in between sentences using embedding similarity. This ensures that a “chunk” contains sentences that are 
# semantically related to each other.
splitter = SemanticSplitterNodeParser(
              buffer_size=1, 
              breakpoint_percentile_threshold=95, 
              embed_model=embed_model
           )

nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/12 [00:00<?, ?it/s]





Parsing nodes:   8%|▊         | 1/12 [00:00<00:05,  1.90it/s]



Parsing nodes:  17%|█▋        | 2/12 [00:01<00:07,  1.31it/s]



Parsing nodes:  25%|██▌       | 3/12 [00:02<00:09,  1.00s/it]









Parsing nodes:  33%|███▎      | 4/12 [00:07<00:19,  2.47s/it]



Parsing nodes:  42%|████▏     | 5/12 [00:08<00:13,  2.00s/it]









Parsing nodes:  50%|█████     | 6/12 [00:14<00:19,  3.24s/it]





Parsing nodes:  58%|█████▊    | 7/12 [00:16<00:14,  2.85s/it]





Parsing nodes:  67%|██████▋   | 8/12 [00:17<00:09,  2.47s/it]







Parsing nodes:  75%|███████▌  | 9/12 [00:20<00:07,  2.51s/it]







Parsing nodes:  83%|████████▎ | 10/12 [00:23<00:05,  2.62s/it]



Parsing nodes:  92%|█████████▏| 11/12 [00:24<00:02,  2.09s/it]





Parsing nodes: 100%|██████████| 12/12 [00:26<00:00,  2.20s/it]


In [26]:
nodes[0]

TextNode(id_='eb08967a-c9b2-4c79-aedd-e02630a998ab', embedding=None, metadata={'page_label': '1', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'documents\\coe_demo\\data-science-lifecycle-ebook.pdf', 'file_type': 'application/pdf', 'file_size': 7899425, 'creation_date': '2024-05-20', 'last_modified_date': '2024-01-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a91546ca-ecbe-4fba-9d20-6688c23f60df', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'data-science-lifecycle-ebook.pdf', 'file_path': 'documents\\coe_demo\\data-science-lifecycle-ebook.pdf', 'file_type': 'application/pdf', 'file_size': 7899425, 'creation_date': '2024-05-20', 'last_modified_date

In [27]:
# Transformations & Ingestion Pipeline
from llama_index.core import Document
# from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        TitleExtractor(),
        CohereEmbedding(),
    ]
)

print(Document.example())
# run the pipeline
nodes = pipeline.run(documents=[Document.example()])
print(nodes)

Generating embeddings:   0%|          | 0/1 [31:21<?, ?it/s]


ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [28]:
from llama_index.llms.groq import Groq
# from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.node_parser import SentenceSplitter

# The Llamaindex Settings module helps in configuring global settings that can be used throughout the pipeline.
from llama_index.core import Settings

Settings.llm = Groq(model="llama3-70b-8192")
Settings.embed_model = CohereEmbedding(model_name="embed-english-v3.0")
Settings.text_splitter = SentenceSplitter(chunk_size=1024)
Settings.chunk_size = 512
Settings.chunk_overlap = 20
Settings.transformations = [SentenceSplitter(chunk_size=1024)]
# maximum input size to the LLM
Settings.context_window = 4096

# number of tokens reserved for text generation.
Settings.num_output = 256

Generating embeddings:  53%|█████▎    | 9/17 [22:21<19:52, 149.11s/it]


In [33]:
llm = Groq(model="llama3-70b-8192", api_key=os.getenv('GROQ_API_KEY'))
llm

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001BCD64C7280>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x000001BCC0232170>, completion_to_prompt=<function default_completion_to_prompt at 0x000001BCC02B6320>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama3-70b-8192', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

AttributeError: 'Groq' object has no attribute 'invoke'